In [6]:
import pandas as pd
import numpy as np

import pandas as pd

def calculate_total_charging_duration(df, min_periods=3):
    # Resample the DataFrame to daily frequency and calculate the sum of consecutive periods
    daily_sum = df.resample('D').sum()
    
    # Filter out charging events without at least the minimum number of consecutive periods
    filtered_sum = daily_sum.mask(daily_sum < min_periods, 0)
    
    # Calculate the total charging duration for each day
    total_duration = filtered_sum.sum(axis=1)
    
    return total_duration


# Generate dummy data
np.random.seed(42)
num_vehicles = 3
start_date = '2023-05-01'
end_date = '2023-05-08'
time_interval = '10T'

dates = pd.date_range(start=start_date, end=end_date, freq=time_interval)
data = np.random.choice([0, 1], size=(len(dates), num_vehicles))

df = pd.DataFrame(data, index=dates, columns=[f'Vehicle {i+1}' for i in range(num_vehicles)])

# Test the calculate_total_charging_duration function
total_charging_duration = calculate_total_charging_duration(df)
print(total_charging_duration)


2023-05-01    221
2023-05-02    219
2023-05-03    205
2023-05-04    209
2023-05-05    206
2023-05-06    219
2023-05-07    219
2023-05-08      0
Freq: D, dtype: int64


In [7]:
df.to_csv('NREL-TESTING.csv')